In [77]:
import pandas as pd 

In [78]:
# df = pd.read_csv('/Users/shuxin/projects/market_analysis/LLM_market_selection/V2/data/20250802_all_10w_with_rank_ratio.csv')
# df.rename(columns={df.columns[0]: 'keyword'}, inplace=True)
# new_df = df[(df['price_avg'] >= 20) & (df['m1_mid'] >= 0.50)]
# print(f"主数据筛选后数量: {len(new_df)}")

# df2 = pd.read_csv('/Users/shuxin/projects/market_analysis/LLM_market_selection/NR/new_release_20250830.csv')
# new_df2 = df2[(df2['price_avg'] >= 15) & (df2['m1_mid'] >= 0.45)]

# merged_df = pd.concat([
#     new_df[~new_df['keyword'].isin(new_df2['keyword'])],
#     new_df2
# ], ignore_index=True)
# print(f"合并去重后总数量: {len(merged_df)}")

In [80]:
# 按照要求筛选数据：price_avg >= 18 且 m1_mid > 0.55
# new_df = df[((df['price_avg'] >= 20) & (df['m1_mid'] >= 0.50)) | (df['relative_available_date_days_mid'] < 365)]
import os 
import time 

# 在jupyter notebook中获取当前工作目录
current_dir = os.getcwd()

folder_path = '/Users/shuxin/projects/market_analysis/LLM_market_selection/V2/'
file_path = 'data/20250913_part_1_with_rank_ratio.csv'
# 定义中间结果保存文件名

file_name = os.path.splitext(os.path.basename(file_path))[0]
temp_output_file = current_dir + f'/data/{file_name}_material_analysis_results_temp_{int(time.time())}.csv'
output_file = current_dir + '/data/' + file_name + '_material_analysis_results.csv'
image_save_dir = current_dir + f"/images/downloaded_images_{file_name}"

def get_df():
    df= all_word = pd.read_csv(folder_path + file_path)
    df = df.rename(columns={all_word.columns[0]: 'keyword'})
    new_df = df[((df['price_avg'] >= 20) 
              & (df['m1_mid'] >= 0.50) 
              # & (df['cr_by_product_cr5'] <= 0.8) 
                # & (df['score'] >= 6.5) 
              )]
    print(len(new_df))
    return new_df

merged_df = get_df() 

3348


In [81]:
import base64
import os
import imghdr

def encode_image(image_path):
    """
    将图片编码为base64格式，并返回data URI格式的字符串
    """
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
        # 检测图片类型
        img_type = imghdr.what(None, image_data)
        if img_type is None:
            img_type = image_path.split('.')[-1]
        # 转换为base64并拼接data URI
        base64_image = base64.b64encode(image_data).decode('utf-8')
        return f"data:image/{img_type};base64,{base64_image}"

def find_images_by_keyword(keyword, download_images_dir=image_save_dir):
    """
    根据关键词查找以 keyword_ 开头的图片
    返回图片的base64编码
    """
    base64_images = []
    # 确保目录路径存在
    if not os.path.exists(download_images_dir):
        return []
    prefix = f"{keyword}_"
    # 遍历目录下的所有文件
    for filename in os.listdir(download_images_dir):
        # 检查文件是否以关键词_开头，且是图片文件
        if filename.startswith(prefix) and any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            image_file_path = os.path.join(download_images_dir, filename)
            base64_images.append(encode_image(image_file_path))
            
    return base64_images


In [82]:
import pandas as pd  

from typing import List
from pydantic import BaseModel
# 测试调用方舟模型，给定schema，输出json 
from volcenginesdkarkruntime import Ark

# 初始化方舟SDK客户端
client = Ark(
    # 从环境变量获取方舟API Key（需提前设置环境变量）
    api_key="29549de0-26ea-4e17-b73f-09ecdf08b678",
)

# 产品材质分析模型
class ProductMaterialAnalysis(BaseModel):
    easy_to_develop_reason: str  # 容易开发的理由
    easy_to_develop_conclusion: str  # 是否容易开发该产品，不需要开模就可以低成本快速改造的
    material_reason: str  # 理由
    material_conclusion: str   # 最终的判断结果

prompt = """
    你是一位专业的产品材质分析师，现在需要你根据一个关键词对应的市场下的几个产品的图片，以及产品的标题判断材质以及改造生产难度

    # 输入数据
        1. 关键词
        2. 前5个产品的标题
        3. 关键词对应的市场的一级分类
        4. 二级分类
        5. 三级分类
        6. 5个产品的图片

    # 主要工作
    1. 判断该市场的产品的材质。材质需要为金属、陶瓷、玻璃、塑料、橡胶、布料纤维、纸张、骨头、木材、皮革、硅胶、石头珠宝、液体、碳/石墨材料中的一种，如果不属于其中的材质，请输出你认为的材质，如果是电子电器产品不需要判断材质，结果判断为电子产品。
    2. 判断产品是否容易进行差异化改造的，从生产的角度判断是否容易进行低成本的差异化改造和生产。如3C电子电器产品需要开模会比较贵，但是陶瓷、玻璃、木材、纸张等材质的产品容易进行低成本的差异化改造。
    3. 给出最终的判断结果。
    
    # 输出
    easy_to_develop_reason # 是否容易开发该产品，不需要复杂的生产流程就可以低成本快速改造的理由
    easy_to_develop_conclusion # 是否容易开发该产品的结论
    material_reason # 材质判断的理由
    material_conclusion # 最终的材质判断结果

    所有输出必须为中文。
"""


def get_response(item, schema=ProductMaterialAnalysis):
    keyword = item['keyword']
    images = find_images_by_keyword(keyword)
    top5_title = item['top5_title'].split(';;')
    common_category_l1 = item['common_category_l1']
    common_category_l2 = item['common_category_l2']
    common_category_l3 = item['common_category_l3']

    user_msg = [
        {
            "type": "text",
            "text": f"关键词：{keyword}\n前5个产品的标题分别为：{top5_title}\n关键词对应的市场的一级分类：{common_category_l1}\n二级分类：{common_category_l2}\n三级分类：{common_category_l3} 其中5个产品的图片分别为："
        }
    ]
    for image in images:
        user_msg.append({
            "type": "image_url",
            "image_url": {
                "url":  image
            },         
        })
    user_msg.append({
            "type": "text",
            "text": "请根据以上图片和文字，判断该市场的产品的材质以及理由。先给出理由，才给出最终的材质结果。"
    })
    
    # 调用方舟模型生成响应（自动解析为指定模型），并强制要求返回中文
    completion = client.beta.chat.completions.parse(
        model="doubao-seed-1-6-250615",  # 具体模型需替换为实际可用模型
        messages=[
            {"role": "system", "content": prompt},
            {
                "role": "user", 
                "content": user_msg
            }
        ],
        response_format=schema,  # 指定响应解析模型
        extra_body={
            "thinking": {
                "type": "disabled" # 不使用深度思考能力
            }
        }
    )
    # 提取解析后的结构化响应
    resp = completion.choices[0].message.parsed
    return resp 

In [ ]:
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import pandas as pd
import os
import time
# Define kept columns
kept_cols = ['keyword', 'top5_images', 'top5_title', 'common_category_l1', 'common_category_l2', 'common_category_l3']

# Prepare result DataFrame
result_df = merged_df[kept_cols].copy()
result_df['easy_to_develop_reason'] = ''
result_df['easy_to_develop_conclusion'] = ''
result_df['material_reason'] = ''
result_df['material_conclusion'] = ''



def process_row(args):
    index, row = args
    try:
        item = row.to_dict()
        resp = get_response(item)  # Assumes get_response is defined
        return {
            'index': index,
            'easy_to_develop_reason': resp.easy_to_develop_reason,
            'easy_to_develop_conclusion': resp.easy_to_develop_conclusion,
            'material_reason': resp.material_reason,
            'material_conclusion': resp.material_conclusion
        }
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        return {
            'index': index,
            'easy_to_develop_reason': 'Error',
            'easy_to_develop_conclusion': 'Error',
            'material_reason': 'Error',
            'material_conclusion': 'Error'
        }

# Process rows using ThreadPoolExecutor
max_workers = 4  # Adjust based on system and workload
results = []
processed_count = 0
save_interval = 10  # 每处理10条数据保存一次

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for args in merged_df.iterrows():
        future = executor.submit(process_row, args)
        futures.append(future)
    
    # 使用tqdm显示进度
    for future in tqdm(futures, total=len(merged_df), desc="Processing rows"):
        result = future.result()
        results.append(result)
        processed_count += 1
        
        # 更新DataFrame并定期保存
        index = result['index']
        result_df.loc[index, 'easy_to_develop_reason'] = result['easy_to_develop_reason']
        result_df.loc[index, 'easy_to_develop_conclusion'] = result['easy_to_develop_conclusion']
        result_df.loc[index, 'material_reason'] = result['material_reason']
        result_df.loc[index, 'material_conclusion'] = result['material_conclusion']
        
        if processed_count % save_interval == 0:
            # 保存中间结果
            result_df.to_csv(temp_output_file, index=False)
            print(f"Saved intermediate results after processing {processed_count} items")

# Save final results to CSV

result_df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# 删除临时文件
if os.path.exists(temp_output_file):
    os.remove(temp_output_file)


In [84]:
material_analysis_result = pd.read_csv(output_file)

In [86]:
x = pd.read_csv('/Users/shuxin/projects/market_analysis/LLM_market_selection/V2/data/20250913_part_1-merged_features.csv')

# 将material_analysis_result和x按keyword列进行合并
merged_df = pd.merge(material_analysis_result, x, on='keyword', how='inner')

In [87]:
keep_cols = [
    'keyword',
    'top5_images',
    'score',
    'conclusion',
    'advantage',
    'risk',
    'common_category_l1_x',
    'common_category_l2_x',
    'common_category_l3_x',
    'easy_to_develop_reason',
    'easy_to_develop_conclusion',
    # 'material_reason',
    'material_conclusion',
    'm1_mid',
    'm2_avg',
    'm2_mid',
    'm2_max',
    'search_rank',
    'purchace',
    'products',
    'ad_products',
    'supply_demand_ratio',
    'ad_supply_demand_ratio',
    'monopoly_asin_dtos_0_click_rate',
    'monopoly_asin_dtos_0_purchase_rate',
    'monopoly_asin_dtos_1_click_rate',
    'monopoly_asin_dtos_1_purchase_rate',
    'monopoly_asin_dtos_2_click_rate',
    'monopoly_asin_dtos_2_purchase_rate',
    'monopoly_click_sum',
    'monopoly_conversion_sum',
    'spr',
    'price_avg',
    'reviews_avg',
    'rating_avg',
    'relative_available_date_days_avg',
    'relative_available_date_days_mid',
    'relative_available_date_days_min',
    'relative_available_date_days_max',
    'weight_mid',
    'volume_mid',
    'top3_sells',
    'top3_amount',
]
# 只保留merged_df中top5_images列的第一个URL
merged_df['top5_images'] = merged_df['top5_images_x'].apply(lambda x: x.split(';;')[0] if pd.notna(x) else x)
# 将top3_sells拆分为三列
sells_split = merged_df['top3_sells'].str.split(';;', expand=True)
merged_df['top1_sells'] = sells_split[0]
merged_df['top2_sells'] = sells_split[1] 
merged_df['top3_sells'] = sells_split[2]

# 更新keep_cols列表
keep_cols.remove('top3_sells')
keep_cols.extend(['top1_sells', 'top2_sells', 'top3_sells'])
merged_df[merged_df.score >= 5.5][keep_cols].to_csv('data/0913_part_1_full.csv',index=False)

In [52]:
# # 对每个三级品类，保留score最高的前40%的数据
# # 添加检查步骤确保过滤后的数据量更小
# original_size = len(merged_df)
# filtered_df = merged_df.groupby('common_category_l3_x').apply(
#     lambda x: x.nlargest(int(len(x)), 'score')
# ).reset_index(drop=True)[keep_cols]

/var/folders/qk/mf0qlk8d141cxwxc748q0bd00000gn/T/ipykernel_35779/2771167973.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = merged_df.groupby('common_category_l3_x').apply(
